# Buying Groups: Causal Targeting to Maximize Profit

#### Buying Groups: Causal Targeting to Maximize Profit

What this script does:
1) Simulates a member–supplier environment with confounding.
2) Computes a naïve diff-in-means (biased).
3) Estimates an ATE with IPTW (sanity check).
4) Trains a simple T-learner to get individual uplift (CATE).
5) Converts uplift to expected PROFIT lift and selects who to treat.
6) Compares policies: Treat None / Treat All / Naïve / Causal Targeting.

Tweak: gross_margin, incentive_cost, or add a budget.

In [10]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

np.random.seed(25)

In [22]:
# -----------------------------
# 1) Create covariates
# -----------------------------
n = 6000

member_size             = np.random.lognormal(mean=2.3, sigma=0.5, size=n)         # ~size proxy
hist_volume             = np.random.lognormal(mean=2.0, sigma=0.6, size=n)         # past volume
supplier_price_idx      = np.random.normal(loc=110, scale=12, size=n)              # 100 ~ market avg
portfolio_overlap       = np.clip(np.random.beta(a=2.5, b=2.5, size=n), 0, 1)      # 0–1
relationship_score      = np.clip(np.random.beta(a=2.0, b=3.0, size=n), 0, 1)      # 0–1
market_bundle_intensity = np.clip(np.random.beta(a=3.0, b=2.0, size=n), 0, 1)      # 0–1

df = pd.DataFrame({
    "member_size": member_size,
    "hist_volume": hist_volume,
    "supplier_price_idx": supplier_price_idx,
    "portfolio_overlap": portfolio_overlap,
    "relationship_score": relationship_score,
    "market_bundle_intensity": market_bundle_intensity
})

# -----------------------------
# 2) Treatment assignment (propensity -> T)
# -----------------------------
lin_T = (
    0.40 * stats.zscore(df["member_size"]) +
    0.45 * stats.zscore(df["hist_volume"]) +
    0.35 * stats.zscore(df["portfolio_overlap"]) +
    0.25 * stats.zscore(df["relationship_score"]) +
    0.30 * stats.zscore(df["market_bundle_intensity"]) -
    0.40 * stats.zscore(df["supplier_price_idx"]) +
    np.random.normal(0, 0.5, n)
)
ps_true = 1 / (1 + np.exp(-lin_T))
df["T"] = np.random.binomial(1, ps_true, size=n)
df["ps_true"] = ps_true

# ============================================
# 3) Supplier economics + revenue/profit model
# ============================================
gross_margin       = 0.35     # supplier gross margin on revenue
incentive_cost     = 5.00     # cost per treated member
tau_revenue_true   = 12.0     # causal revenue lift ($)

# Log-link baseline so revenue is positive
lin_rev = (
    0.85 * stats.zscore(df["member_size"]) +
    0.90 * stats.zscore(df["hist_volume"])  +
   -0.60 * stats.zscore(df["supplier_price_idx"]) +
    0.45 * stats.zscore(df["portfolio_overlap"]) +
    0.40 * stats.zscore(df["relationship_score"]) +
    0.35 * stats.zscore(df["market_bundle_intensity"])
)

mu_log    = 4.8            # exp(4.8) ≈ 121 → typical untreated revenue around $120–$150
sigma_log = 0.30

y0_revenue = np.exp(mu_log + lin_rev + np.random.normal(0, sigma_log, size=n))  # untreated revenue > 0
y1_revenue = y0_revenue + tau_revenue_true                                      # additive lift (use * for multiplicative)

# Convert revenue → profit (supplier perspective)
y0_profit = gross_margin * y0_revenue                     # control: no incentive cost
y1_profit = gross_margin * y1_revenue - incentive_cost    # treated: pay incentive

# Realized profit by observed treatment
df["y_profit"] = np.where(df["T"] == 1, y1_profit, y0_profit)

# ---------------------------------
# 4) Sanity checks / diagnostics
# ---------------------------------
m1 = df["T"] == 1
m0 = ~m1

neg_control_share = (df.loc[m0, "y_profit"] < 0).mean()
neg_treated_share = (df.loc[m1, "y_profit"] < 0).mean()

print("== Profit Sanity Checks ==")
print(f"Negative profit (Control, T=0): {100*neg_control_share:.2f}%")
print(f"Negative profit (Treated, T=1): {100*neg_treated_share:.2f}%")
print(f"Profit summary (Control) → min: {df.loc[m0,'y_profit'].min():.2f}, median: {df.loc[m0,'y_profit'].median():.2f}")
print(f"Profit summary (Treated) → min: {df.loc[m1,'y_profit'].min():.2f}, median: {df.loc[m1,'y_profit'].median():.2f}")

== Profit Sanity Checks ==
Negative profit (Control, T=0): 0.00%
Negative profit (Treated, T=1): 0.03%
Profit summary (Control) → min: 0.79, median: 22.26
Profit summary (Treated) → min: -0.10, median: 60.23


In [23]:
df

,member_size,hist_volume,supplier_price_idx,portfolio_overlap,relationship_score,market_bundle_intensity,T,ps_true,y_profit
0,11.180113,1.985542,109.658305,0.328587,0.528137,0.468916,0,0.214661,6.664785
1,16.667241,9.061417,113.905562,0.236058,0.355669,0.520590,0,0.344369,17.952424
2,6.554865,3.504027,116.750218,0.335776,0.421876,0.883297,0,0.253308,4.676975
3,7.421708,14.194127,115.878187,0.391992,0.727685,0.705441,1,0.490858,102.843612
4,6.181468,15.803745,106.040371,0.579616,0.233268,0.350438,0,0.399961,60.652737
...,...,...,...,...,...,...,...,...,...
5995,27.978041,3.521359,113.840321,0.126901,0.390761,0.785667,0,0.473333,97.989091
5996,5.172679,4.715986,111.568193,0.555272,0.349225,0.846986,0,0.561535,8.748845
5997,8.126426,13.579272,103.134369,0.374922,0.443655,0.547224,0,0.299663,41.339108
5998,17.216038,7.676219,108.328572,0.613947,0.635691,0.634752,0,0.644101,156.207043


In [24]:
counts = df['T'].value_counts()
print(counts)

T
1    3033
0    2967
Name: count, dtype: int64


In [25]:
# ====================================
# 2) Naïve difference in means (biased)
# ====================================
naive_diff_profit = df.loc[df["T"]==1, "y_profit"].mean() - df.loc[df["T"]==0, "y_profit"].mean()
print("== Naïve Comparison on PROFIT (Unadjusted) ==")
print(f"Mean(profit | T=1) - Mean(profit | T=0) = {naive_diff_profit:,.2f}")
# Note: Still confounded by targeting (larger/richer/high-overlap members more likely to get offers)

== Naïve Comparison on PROFIT (Unadjusted) ==
Mean(profit | T=1) - Mean(profit | T=0) = 1,130.93


In [27]:
# ==================================================
# 3) IPTW ATE (propensity-weighted) on PROFIT
# ==================================================
features = ["member_size","hist_volume","supplier_price_idx",
            "portfolio_overlap","relationship_score","market_bundle_intensity"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

logit = LogisticRegression(max_iter=400, solver="lbfgs")
logit.fit(X_scaled, df["T"])
ps_hat = np.clip(logit.predict_proba(X_scaled)[:, 1], 1e-3, 1-1e-3)
df["ps"] = ps_hat

# IPTW weights
w = np.where(df["T"]==1, 1/df["ps"], 1/(1-df["ps"]))
df["w"] = w

m1 = df["T"]==1
m0 = ~m1

y1_hat_iptw = np.average(df.loc[m1,"y_profit"].to_numpy(), weights=df.loc[m1,"w"].to_numpy())
y0_hat_iptw = np.average(df.loc[m0,"y_profit"].to_numpy(), weights=df.loc[m0,"w"].to_numpy())
ate_iptw_profit = y1_hat_iptw - y0_hat_iptw

# Define the true profit effect
tau_profit_true = gross_margin * tau_revenue_true - incentive_cost

print("== IPTW ATE on PROFIT (sanity check) ==")
print(f"True profit effect (tau_profit_true): {tau_profit_true:,.2f}")
print(f"IPTW ATE (est., profit):              {ate_iptw_profit:,.2f}\n")

== IPTW ATE on PROFIT (sanity check) ==
True profit effect (tau_profit_true): -0.80
IPTW ATE (est., profit):              482.72



In [28]:
# ==========================================
# 4) T-learner on PROFIT (heterogeneous CATE)
# ==========================================
X_train, X_test, y_train, y_test, T_train, T_test = train_test_split(
    df[features], df["y_profit"], df["T"], test_size=0.3, random_state=25, stratify=df["T"]
)

mod_treated = GradientBoostingRegressor(random_state=25)
mod_control = GradientBoostingRegressor(random_state=25)

mod_treated.fit( X_train[T_train==1],  y_train[T_train==1] )
mod_control.fit( X_train[T_train==0],  y_train[T_train==0] )

mu1_hat_profit = mod_treated.predict(df[features])
mu0_hat_profit = mod_control.predict(df[features])
cate_hat_profit = mu1_hat_profit - mu0_hat_profit
df["cate_hat_profit"] = cate_hat_profit

df

,member_size,hist_volume,supplier_price_idx,portfolio_overlap,relationship_score,market_bundle_intensity,T,ps_true,y_profit,ps,w,cate_hat_profit
0,11.180113,1.985542,109.658305,0.328587,0.528137,0.468916,0,0.214661,6.664785,0.314915,1.459672,3.502697
1,16.667241,9.061417,113.905562,0.236058,0.355669,0.520590,0,0.344369,17.952424,0.403333,1.675976,-8.326421
2,6.554865,3.504027,116.750218,0.335776,0.421876,0.883297,0,0.253308,4.676975,0.340508,1.516318,7.455698
3,7.421708,14.194127,115.878187,0.391992,0.727685,0.705441,1,0.490858,102.843612,0.593497,1.684928,-72.161352
4,6.181468,15.803745,106.040371,0.579616,0.233268,0.350438,0,0.399961,60.652737,0.479656,1.921805,5.404395
...,...,...,...,...,...,...,...,...,...,...,...,...
5995,27.978041,3.521359,113.840321,0.126901,0.390761,0.785667,0,0.473333,97.989091,0.530695,2.130810,1244.413144
5996,5.172679,4.715986,111.568193,0.555272,0.349225,0.846986,0,0.561535,8.748845,0.435637,1.771909,14.509926
5997,8.126426,13.579272,103.134369,0.374922,0.443655,0.547224,0,0.299663,41.339108,0.545998,2.202633,-33.744573
5998,17.216038,7.676219,108.328572,0.613947,0.635691,0.634752,0,0.644101,156.207043,0.705574,3.396444,-42.567845


In [29]:
# ==================================================
# 5) Policy: treat if EXPECTED PROFIT LIFT > 0
# ==================================================
policy_causal = (df["cate_hat_profit"] > 0).astype(int)

# Naïve policy baseline (treat same share but choose top hist_volume)
treat_rate = policy_causal.mean()
k = int(round(treat_rate * n))
idx_naive = df["hist_volume"].sort_values(ascending=False).index[:k]
policy_naive = pd.Series(0, index=df.index)
policy_naive.loc[idx_naive] = 1
policy_naive = policy_naive.astype(int)

policy_all  = pd.Series(1, index=df.index)
policy_none = pd.Series(0, index=df.index)

In [30]:
# ==================================================
# 6) Evaluate policies using known potential outcomes in PROFIT
# ==================================================
def policy_profit(policy_vector):
    # potential profits
    prof0 = y0_profit
    prof1 = y1_profit
    # realized profit under policy
    prof_real = np.where(policy_vector.values==1, prof1, prof0)
    return prof_real.mean(), policy_vector.mean()

results = []
for name, pol in [
    ("Treat None", policy_none),
    ("Treat All",  policy_all),
    ("Naïve Targeting (top hist_volume)", policy_naive),
    ("Causal Targeting (uplift>0 on profit)", policy_causal),
]:
    avg_profit, treat_share = policy_profit(pol)
    results.append([name, avg_profit, treat_share])

res_df = pd.DataFrame(results, columns=["Policy", "Avg Profit per Member", "Treat Share"]).sort_values(
    by="Avg Profit per Member", ascending=False
)

print("== Policy Comparison (SUPPLIER PROFIT per Member) ==")
print(res_df.to_string(index=False, formatters={
    "Avg Profit per Member": lambda v: f"${v:,.2f}",
    "Treat Share":           lambda v: f"{100*v:,.1f}%"
}))
print("\nNotes:")
print("- Now the outcome is PROFIT, not revenue.")
print("- Causal Targeting treats only when expected profit lift > 0.")
print("- Naïve Targeting treats the same share but by a crude heuristic (top hist_volume).\n")

best = res_df.iloc[0]
print("== Takeaway (Supplier Profit) ==")
print(f"Best policy: {best['Policy']} | Profit/member = {best['Avg Profit per Member']:.2f} "
      f"| Treat share = {best['Treat Share']:.1%}")

== Policy Comparison (SUPPLIER PROFIT per Member) ==
                               Policy Avg Profit per Member Treat Share
                           Treat None               $645.79        0.0%
Causal Targeting (uplift>0 on profit)               $645.35       53.9%
    Naïve Targeting (top hist_volume)               $645.35       53.9%
                            Treat All               $644.99      100.0%

Notes:
- Now the outcome is PROFIT, not revenue.
- Causal Targeting treats only when expected profit lift > 0.
- Naïve Targeting treats the same share but by a crude heuristic (top hist_volume).

== Takeaway (Supplier Profit) ==
Best policy: Treat None | Profit/member = 645.79 | Treat share = 0.0%


In [31]:
# ============================================
# 1) Create covariates
# ============================================
np.random.seed(25)  # Reproducibility
n = 6000

member_size             = np.random.lognormal(mean=2.3, sigma=0.5, size=n)         # size proxy
hist_volume             = np.random.lognormal(mean=2.0, sigma=0.6, size=n)         # past volume
supplier_price_idx      = np.random.normal(loc=110, scale=12, size=n)              # 100 ~ market avg
portfolio_overlap       = np.clip(np.random.beta(a=2.5, b=2.5, size=n), 0, 1)      # 0–1
relationship_score      = np.clip(np.random.beta(a=2.0, b=3.0, size=n), 0, 1)      # 0–1
market_bundle_intensity = np.clip(np.random.beta(a=3.0, b=2.0, size=n), 0, 1)      # 0–1

df = pd.DataFrame({
    "member_size": member_size,
    "hist_volume": hist_volume,
    "supplier_price_idx": supplier_price_idx,
    "portfolio_overlap": portfolio_overlap,
    "relationship_score": relationship_score,
    "market_bundle_intensity": market_bundle_intensity
})

# ============================================
# 2) Treatment assignment (propensity -> T)
#    Larger size, higher volume/overlap/relationship/market intensity => higher P(T=1)
#    Pricier suppliers => lower P(T=1)
# ============================================
lin_T = (
    0.40 * stats.zscore(df["member_size"]) +
    0.45 * stats.zscore(df["hist_volume"]) +
    0.35 * stats.zscore(df["portfolio_overlap"]) +
    0.25 * stats.zscore(df["relationship_score"]) +
    0.30 * stats.zscore(df["market_bundle_intensity"]) -
    0.40 * stats.zscore(df["supplier_price_idx"]) +
    np.random.normal(0, 0.5, n)  # targeting noise
)
ps_true = 1 / (1 + np.exp(-lin_T))
df["T"] = np.random.binomial(1, ps_true, size=n)
df["ps_true"] = ps_true

# ============================================
# 3) Supplier economics + revenue/profit model
#    Choose parameters so treated/control profits are non-negative
# ============================================
gross_margin       = 0.35     # supplier gross margin on revenue
incentive_cost     = 4.00     # cost per treated member
tau_revenue_true   = 20.0     # causal revenue lift ($)

# True profit effect (constant, by design)
tau_profit_true = gross_margin * tau_revenue_true - incentive_cost  # = 0.35*20 - 4 = +3.0

# Log-link baseline so revenue is positive and in a realistic range
lin_rev = (
    0.85 * stats.zscore(df["member_size"]) +
    0.90 * stats.zscore(df["hist_volume"])  -
    0.60 * stats.zscore(df["supplier_price_idx"]) +
    0.45 * stats.zscore(df["portfolio_overlap"]) +
    0.40 * stats.zscore(df["relationship_score"]) +
    0.35 * stats.zscore(df["market_bundle_intensity"])
)
mu_log    = 4.7      # exp(4.7) ~ 110 → typical untreated revenue ~ $100–$160
sigma_log = 0.25     # moderate dispersion

# Untreated revenue (>0 by construction)
y0_revenue = np.exp(mu_log + lin_rev + np.random.normal(0, sigma_log, size=n))

# Treated revenue: additive lift (keeps tau_profit_true constant)
y1_revenue = y0_revenue + tau_revenue_true

# Convert revenue → profit (supplier perspective)
# Control: no incentive cost; Treated: pay incentive
y0_profit = gross_margin * y0_revenue
y1_profit = gross_margin * y1_revenue - incentive_cost

# Realized profit by observed treatment
df["y_profit"] = np.where(df["T"] == 1, y1_profit, y0_profit)

# ---------------------------------
# 4) Sanity checks / diagnostics
# ---------------------------------
m1 = df["T"] == 1
m0 = ~m1

neg_control_share = (df.loc[m0, "y_profit"] < 0).mean()
neg_treated_share = (df.loc[m1, "y_profit"] < 0).mean()

print("== Profit Sanity Checks ==")
print(f"Negative profit (Control, T=0): {100*neg_control_share:.2f}%")   # expected ~0%
print(f"Negative profit (Treated, T=1): {100*neg_treated_share:.2f}%")   # expected ~0%
print(f"Profit summary (Control) → min: {df.loc[m0,'y_profit'].min():.2f}, median: {df.loc[m0,'y_profit'].median():.2f}")
print(f"Profit summary (Treated) → min: {df.loc[m1,'y_profit'].min():.2f}, median: {df.loc[m1,'y_profit'].median():.2f}")

# ============================================
# 5) IPTW ATE on PROFIT (should recover ~ +3.0)
# ============================================
features = ["member_size","hist_volume","supplier_price_idx",
            "portfolio_overlap","relationship_score","market_bundle_intensity"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

logit = LogisticRegression(max_iter=400, solver="lbfgs")
logit.fit(X_scaled, df["T"])
ps_hat = np.clip(logit.predict_proba(X_scaled)[:, 1], 1e-3, 1-1e-3)
df["ps"] = ps_hat

# IPTW weights
w = np.where(df["T"]==1, 1/df["ps"], 1/(1-df["ps"]))
df["w"] = w

# Weighted means
y1_hat_iptw = np.average(df.loc[m1,"y_profit"].to_numpy(),
                         weights=df.loc[m1,"w"].to_numpy())
y0_hat_iptw = np.average(df.loc[m0,"y_profit"].to_numpy(),
                         weights=df.loc[m0,"w"].to_numpy())
ate_iptw_profit = y1_hat_iptw - y0_hat_iptw

print("\n== IPTW ATE on PROFIT ==")
print(f"True profit effect (tau_profit_true): {tau_profit_true:,.2f}")
print(f"IPTW ATE (estimated, profit):         {ate_iptw_profit:,.2f}")

# (Optional) quick weight diagnostics to ensure no extreme instability
print("\n== Weight Diagnostics ==")
print(df["w"].describe(percentiles=[0.5, 0.9, 0.95, 0.99]).to_string())

== Profit Sanity Checks ==
Negative profit (Control, T=0): 0.00%
Negative profit (Treated, T=1): 0.00%
Profit summary (Control) → min: 0.76, median: 20.27
Profit summary (Treated) → min: 3.63, median: 57.86

== IPTW ATE on PROFIT ==
True profit effect (tau_profit_true): 3.00
IPTW ATE (estimated, profit):         440.29

== Weight Diagnostics ==
count    6000.000000
mean        1.998838
std         0.920047
min         1.004545
50%         1.742549
90%         2.995548
95%         3.553923
99%         5.783431
max        13.702589


In [ ]:
# ============================================
# Buying Groups — Causal Targeting to Maximize SUPPLIER PROFIT
# ============================================
# What this script does (supplier perspective):
# 1) Simulates confounded data in a buying-group setting.
# 2) Builds outcomes as PROFIT (not revenue):
#       profit = gross_margin * revenue - incentive_cost * T
# 3) Shows naïve diff-in-means (biased), IPTW ATE (sanity check).
# 4) Trains a T-learner to predict profit uplift (CATE on PROFIT).
# 5) Chooses who to treat based on EXPECTED PROFIT LIFT > 0.
# 6) Compares policies (Treat None / Treat All / Naïve / Causal).

import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

np.random.seed(25)

# ==============================
# 0) Supplier economics
# ==============================
gross_margin   = 0.35   # supplier gross margin on incremental revenue
incentive_cost = 5.00   # cost to offer the bundle (per treated member)

# ==============================
# 1) Simulate buying-group data
# ==============================
n = 6000

df = pd.DataFrame({
    "member_size":            np.random.lognormal(mean=2.8,  sigma=0.5,  size=n),
    "hist_volume":            np.random.lognormal(mean=3.1,  sigma=0.55, size=n),
    "supplier_price_idx":     np.random.normal(loc=100,      scale=12,   size=n),
    "portfolio_overlap":      np.clip(np.random.beta(a=2.5, b=1.8, size=n), 0, 1),
    "relationship_score":     np.clip(np.random.normal(loc=0.6, scale=0.2, size=n), 0, 1),
    "market_bundle_intensity":np.clip(np.random.normal(loc=0.5, scale=0.2, size=n), 0, 1),
})

# True treatment effect on member revenue (not profit)
tau_revenue_true = 12.0

# Treatment assignment is confounded (bigger/richer/overlap/relationship/market more likely; high price less likely)
logit_T = (
    0.40 * stats.zscore(df["member_size"]) +
    0.45 * stats.zscore(df["hist_volume"]) +
    0.35 * stats.zscore(df["portfolio_overlap"]) +
    0.25 * stats.zscore(df["relationship_score"]) +
    0.30 * stats.zscore(df["market_bundle_intensity"]) -
    0.40 * stats.zscore(df["supplier_price_idx"]) +
    np.random.normal(0, 0.5, n)
)
ps_true = 1 / (1 + np.exp(-logit_T))
df["T"] = np.random.binomial(1, ps_true, size=n)

# Baseline REVENUE model (no treatment)
baseline_rev = (
    15.0
    + 3.8  * stats.zscore(df["member_size"])
    + 4.2  * stats.zscore(df["hist_volume"])
    - 2.5  * stats.zscore(df["supplier_price_idx"])
    + 2.0  * stats.zscore(df["portfolio_overlap"])
    + 1.8  * stats.zscore(df["relationship_score"])
    + 1.5  * stats.zscore(df["market_bundle_intensity"])
)

# Potential outcomes in REVENUE
y0_revenue = baseline_rev + np.random.normal(0, 5.0, n)              # if not treated
y1_revenue = baseline_rev + tau_revenue_true + np.random.normal(0, 5.0, n)  # if treated

# Convert REVENUE to PROFIT (supplier perspective)
# profit = margin * revenue - incentive_cost * T
y0_profit = gross_margin * y0_revenue - 0.0
y1_profit = gross_margin * y1_revenue - incentive_cost

# Realized PROFIT outcome
df["y_profit"] = np.where(df["T"]==1, y1_profit, y0_profit)

# For reporting, the true average PROFIT lift (if everyone treated vs not treated)
tau_profit_true = (gross_margin * tau_revenue_true) - incentive_cost

# ====================================
# 2) Naïve difference in means (biased)
# ====================================
naive_diff_profit = df.loc[df["T"]==1, "y_profit"].mean() - df.loc[df["T"]==0, "y_profit"].mean()
print("== Naïve Comparison on PROFIT (Unadjusted) ==")
print(f"Mean(profit | T=1) - Mean(profit | T=0) = {naive_diff_profit:,.2f}")
print("Note: Still confounded by targeting (larger/richer/high-overlap members more likely to get offers).\n")

# ==================================================
# 3) IPTW ATE (propensity-weighted) on PROFIT
# ==================================================
features = ["member_size","hist_volume","supplier_price_idx",
            "portfolio_overlap","relationship_score","market_bundle_intensity"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

logit = LogisticRegression(max_iter=400, solver="lbfgs")
logit.fit(X_scaled, df["T"])
ps_hat = np.clip(logit.predict_proba(X_scaled)[:, 1], 1e-3, 1-1e-3)
df["ps"] = ps_hat

w = np.where(df["T"]==1, 1/df["ps"], 1/(1-df["ps"]))
df["w"] = w
m1 = df["T"]==1
m0 = ~m1

y1_hat_iptw = np.average(df.loc[m1,"y_profit"].to_numpy(), weights=df.loc[m1,"w"].to_numpy())
y0_hat_iptw = np.average(df.loc[m0,"y_profit"].to_numpy(), weights=df.loc[m0,"w"].to_numpy())
ate_iptw_profit = y1_hat_iptw - y0_hat_iptw

print("== IPTW ATE on PROFIT (sanity check) ==")
print(f"True profit effect (tau_profit_true): {tau_profit_true:,.2f}")
print(f"IPTW ATE (est., profit):              {ate_iptw_profit:,.2f}\n")

# ==========================================
# 4) T-learner on PROFIT (heterogeneous CATE)
# ==========================================
X_train, X_test, y_train, y_test, T_train, T_test = train_test_split(
    df[features], df["y_profit"], df["T"], test_size=0.3, random_state=25, stratify=df["T"]
)

mod_treated = GradientBoostingRegressor(random_state=25)
mod_control = GradientBoostingRegressor(random_state=25)

mod_treated.fit( X_train[T_train==1],  y_train[T_train==1] )
mod_control.fit( X_train[T_train==0],  y_train[T_train==0] )

mu1_hat_profit = mod_treated.predict(df[features])
mu0_hat_profit = mod_control.predict(df[features])
cate_hat_profit = mu1_hat_profit - mu0_hat_profit
df["cate_hat_profit"] = cate_hat_profit

# ==================================================
# 5) Policy: treat if EXPECTED PROFIT LIFT > 0
# ==================================================
policy_causal = (df["cate_hat_profit"] > 0).astype(int)

# Naïve policy baseline (treat same share but choose top hist_volume)
treat_rate = policy_causal.mean()
k = int(round(treat_rate * n))
idx_naive = df["hist_volume"].sort_values(ascending=False).index[:k]
policy_naive = pd.Series(0, index=df.index)
policy_naive.loc[idx_naive] = 1
policy_naive = policy_naive.astype(int)

policy_all  = pd.Series(1, index=df.index)
policy_none = pd.Series(0, index=df.index)

# ==================================================
# 6) Evaluate policies using known potential outcomes in PROFIT
# ==================================================
def policy_profit(policy_vector):
    # potential profits
    prof0 = y0_profit
    prof1 = y1_profit
    # realized profit under policy
    prof_real = np.where(policy_vector.values==1, prof1, prof0)
    return prof_real.mean(), policy_vector.mean()

results = []
for name, pol in [
    ("Treat None", policy_none),
    ("Treat All",  policy_all),
    ("Naïve Targeting (top hist_volume)", policy_naive),
    ("Causal Targeting (uplift>0 on profit)", policy_causal),
]:
    avg_profit, treat_share = policy_profit(pol)
    results.append([name, avg_profit, treat_share])

res_df = pd.DataFrame(results, columns=["Policy", "Avg Profit per Member", "Treat Share"]).sort_values(
    by="Avg Profit per Member", ascending=False
)

print("== Policy Comparison (SUPPLIER PROFIT per Member) ==")
print(res_df.to_string(index=False, formatters={
    "Avg Profit per Member": lambda v: f"${v:,.2f}",
    "Treat Share":           lambda v: f"{100*v:,.1f}%"
}))
print("\nNotes:")
print("- Now the outcome is PROFIT, not revenue.")
print("- Causal Targeting treats only when expected profit lift > 0.")
print("- Naïve Targeting treats the same share but by a crude heuristic (top hist_volume).\n")

best = res_df.iloc[0]
print("== Takeaway (Supplier Profit) ==")
print(f"Best policy: {best['Policy']} | Profit/member = {best['Avg Profit per Member']:.2f} "
      f"| Treat share = {best['Treat Share']:.1%}")
